<a href="https://colab.research.google.com/github/sattwik-sahu/dse316-hw01/blob/main/src/Q-04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim
from torchvision import datasets, transforms
import torchvision.transforms as transforms
from torchvision.datasets import SVHN
from torch.utils.data import DataLoader, Subset
from torchvision import models
import torch.nn as nn



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
#Transformation for LeNet-5
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4377, 0.4438, 0.4728], std=[0.1980, 0.2010, 0.1970])
])


# Adjusted LeNet-5 for SVHN
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, kernel_size=5, padding=2)  # Adjusted for 3-channel input
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.fc1 = nn.Linear(16*6*6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = torch.tanh(self.conv1(x))
        x = torch.max_pool2d(x, 2)
        x = torch.tanh(self.conv2(x))
        x = torch.max_pool2d(x, 2)
        x = x.view(-1, 16*6*6)
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        x = self.fc3(x)
        return x

# Load the SVHN dataset
#Change the root path
train_dataset = datasets.SVHN(root='/home/moonlab/dl_assign/data', split='train', download=True, transform=transform)
#create subset
subset_indices = np.random.choice(len(train_dataset), len(train_dataset) // 4, replace=False)
dataset_subset = torch.utils.data.Subset(train_dataset, subset_indices)
# Split the dataset
train_size = int(0.8 * len(dataset_subset))
test_size = len(dataset_subset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset_subset, [train_size, test_size])
# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


# Training and evaluation
def train_and_evaluate(model, train_loader, test_loader):
    print('Training and Evaluating the model')
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.003, momentum=0.9)
    for epoch in range(10):
        model.train()
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f'Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}')

        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for data in test_loader:
                images, labels = data[0].to(device), data[1].to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        print(f'Accuracy of the network on the test images: {100 * correct // total} %')


#Saving the model
# def save_model(model, model_name):
#     print(f'Saving the model {model_name}')
#     torch.save(model.state_dict(), f'./SavedModels/{model_name}.pt')


def choose_model():
    print('------------------------------------------')
    print('Training LeNet-5 on SVHN dataset')
    model = LeNet5().to(device)
    train_and_evaluate(model, train_loader, test_loader)
    # save_model(model, 'lenet_svhn')
    print('Finished Training and Evaluating the model')
    print('------------------------------------------')

choose_model()

100%|██████████| 182040794/182040794 [00:04<00:00, 39414514.65it/s]


------------------------------------------
Training LeNet-5 on SVHN dataset
Training and Evaluating the model
Epoch 1, Loss: 2.2512270519306568
Accuracy of the network on the test images: 19 %
Epoch 2, Loss: 2.214317493563656
Accuracy of the network on the test images: 20 %
Epoch 3, Loss: 2.103314858336636
Accuracy of the network on the test images: 32 %
Epoch 4, Loss: 1.7040621198420962
Accuracy of the network on the test images: 52 %
Epoch 5, Loss: 1.2015032200834117
Accuracy of the network on the test images: 69 %
Epoch 6, Loss: 0.8919994810262621
Accuracy of the network on the test images: 76 %
Epoch 7, Loss: 0.7257499212000568
Accuracy of the network on the test images: 79 %
Epoch 8, Loss: 0.6308288539080641
Accuracy of the network on the test images: 80 %
Epoch 9, Loss: 0.5648655469761145
Accuracy of the network on the test images: 81 %
Epoch 10, Loss: 0.517629336061436
Accuracy of the network on the test images: 82 %
Finished Training and Evaluating the model
-------------------

In [ ]:
#Transformation for Alexnet, VGG16 and Resnet
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


# Load the SVHN dataset
#Change the root path
full_dataset = SVHN(root='/home/moonlab/dl_assign/data', split='train', transform=transform, download=True)
subset_indices = np.random.choice(len(full_dataset), len(full_dataset) // 4, replace=False)
dataset_subset = Subset(full_dataset, subset_indices)
# Split the dataset
train_size = int(0.8 * len(dataset_subset))
test_size = len(dataset_subset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset_subset, [train_size, test_size])
#data loader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


# Get the modified model
def get_modified_model(model_name):
    print(f"Loading {model_name}")
    if model_name == 'alexnet':
        model = models.alexnet(pretrained=True)
        model.classifier[6] = nn.Linear(model.classifier[6].in_features, 10)
    elif model_name == 'vgg16':
        model = models.vgg16(pretrained=True)
        model.classifier[6] = nn.Linear(model.classifier[6].in_features, 10)
    elif model_name in ['resnet18', 'resnet50', 'resnet101']:
        model = getattr(models, model_name)(pretrained=True)
        model.fc = nn.Linear(model.fc.in_features, 10)
    else:
        raise ValueError("Unsupported model name")
    model = model.to(device)
    return model


# Training and evaluation
def train_and_evaluate(model, train_loader, test_loader):
    print('Training and Evaluating the model')
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.003)
    for epoch in range(10):
        model.train()
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f'Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}')

        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for data in test_loader:
                images, labels = data[0].to(device), data[1].to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        print(f'Accuracy of the network on the test images: {100 * correct // total} %')

#Saving the model
# def save_model(model, model_name):
#     print(f'Saving the model {model_name}')
#     torch.save(model.state_dict(), f'./SavedModels/{model_name}.pt')


def choose_model():
    model_names = ['alexnet', 'vgg16', 'resnet18', 'resnet50', 'resnet101']
    for model_name in model_names:
        print('-------------------------------------')
        print(f"Training and evaluating {model_name}")
        model = get_modified_model(model_name)
        train_and_evaluate(model, train_loader, test_loader)
        # save_model(model, model_name)
        print('Finished Training and Evaluating the model')
        print('-------------------------------------')

choose_model()

Using downloaded and verified file: /home/moonlab/dl_assign/data/train_32x32.mat
-------------------------------------
Training and evaluating alexnet
Loading alexnet
Training and Evaluating the model
Epoch 1, Loss: 3.4699016783435273
Accuracy of the network on the test images: 18 %
Epoch 2, Loss: 2.2387172717714936
Accuracy of the network on the test images: 18 %
Epoch 3, Loss: 2.238653300630994
Accuracy of the network on the test images: 18 %
Epoch 4, Loss: 2.2367682321623423
Accuracy of the network on the test images: 18 %
Epoch 5, Loss: 2.237156054859078
Accuracy of the network on the test images: 18 %
